# Example: Read 1 Degree Sea Ice (CICE6) Replay Data, Store to Zar

Example using [CICE6Dataset](generated/ufs2arco.CICE6Dataset.rst) to read data from two cycles of the replay dataset.

In [1]:
import xarray as xr
from datetime import datetime, timedelta

In [2]:
import sys
sys.path.append("..")
from ufs2arco import CICE6Dataset, RegridCICE6

## Setup path to read from

In this case, read replay data from the s3 bucket.
Right now, [CICE6Dataset](generated/ufs2arco.CICE6Dataset.rst) requires a `path_in` callable with 3 entries, so that it can build file paths for a given date (denoting a DA cycle), forecast hours to grab, and file prefixes.

In [3]:
def seaice_path(cycles, forecast_hours, file_prefixes):

    upper = "s3://noaa-ufs-gefsv13replay-pds/1deg"
    cycles = [cycles] if not isinstance(cycles, list) else cycles

    files = []
    for cycle in cycles:
        this_dir = f"{cycle.year:04d}/{cycle.month:02d}/{cycle.year:04d}{cycle.month:02d}{cycle.day:02d}{cycle.hour:02d}"
        for fp in file_prefixes:
            for fhr in forecast_hours:
                fhr_delta = timedelta(hours=-9+fhr)
                this_date = cycle+fhr_delta
                this_file = f"{this_dir}/{fp}{this_date.year:04d}-{this_date.month:02d}-{this_date.day:02d}-{this_date.hour*3600:d}.nc"
                files.append(this_file)
    return [f"{upper}/{this_file}" for this_file in files]

For instance to grab the `ocn_` files at forecast hour `fhr00` 
from the DA cycle at 00:00 Jan 1, 1994, this would be used as follows:

In [4]:
cycles = [datetime(1994,1,1,0), datetime(1994,1,1,6)]
seaice_path(cycles, [0], ["iceh_ic."])

['s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/iceh_ic.1993-12-31-54000.nc',
 's3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010106/iceh_ic.1993-12-31-75600.nc']

However, reading from s3 is super slow with `xarray.open_dataset`.
Luckily we can tell fsspec to cache the files locally before opening, this is denoted by
prepending the names as follows.

See discussion [here](https://discourse.pangeo.io/t/reading-goes-r-s3-netcdfs-from-an-aws-ec2-instance-is-it-possible-to-get-faster-speeds-than-from-my-local-machine/2440/13)
for more info.

In [5]:
def cached_path(cycles, forecast_hours, file_prefixes):
    return [f"simplecache::{u}" for u in seaice_path(cycles, forecast_hours, file_prefixes)]

In [6]:
cached_path(cycles, [0], ["iceh_ic."])

['simplecache::s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/iceh_ic.1993-12-31-54000.nc',
 'simplecache::s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010106/iceh_ic.1993-12-31-75600.nc']

## Use CICE6Dataset

Some of that is wrapped up under the hood in [CICE6Dataset](generated/ufs2arco.CICE6Dataset.rst), we just need to give two inputs:

1. The filename mapping as defined above

2. A configuration yaml file

In [7]:
reader = CICE6Dataset(path_in=cached_path, config_filename="config-replay.yaml")

CICE6Dataset.__init__: Could not find coords_path_out in config-replay.yaml, using default.


We'll use this reader to open a dataset with [.open_dataset()](generated/ufs2arco.MOM6Dataset.open_dataset.rst), note that the `fsspec_kwargs` is not necessary for local file reads, but necessary for reading from buckets like s3

In [10]:
%%time
ds = reader.open_dataset(cycles, fsspec_kwargs={"s3":{"anon":True}})
ds

CPU times: user 383 ms, sys: 47.6 ms, total: 430 ms
Wall time: 386 ms


<xarray.Dataset>
Dimensions:         (time: 2, nj: 320, ni: 360, NCAT: 5, VGRDa: 5)
Coordinates: (12/28)
  * time            (time) datetime64[ns] 1993-12-31T15:00:00 1993-12-31T21:0...
    TLON            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    TLAT            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ULON            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ULAT            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    NLON            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ...              ...
    dxe             (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    dye             (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ANGLE           (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ANGLET          (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    cftime          (time) object 1993-12-31 15:00:00 1993-12-31 21:00:00
    ftime           (time) timedelta64[ns] -1 days +15:00:00 -1 days +15:00:00
Dimensions without coordinates: nj, ni
Data variables: (12/177)
    hi              (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    hs              (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    Tsfc            (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    aice            (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    uvel            (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    vvel            (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    ...              ...
    ipond_d         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    apeff_d         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    apond_h         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    hpond_h         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    ipond_h         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    apeff_h         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
Attributes:
    title:        sea ice model output for CICE
    contents:     Diagnostic and Prognostic Variables
    source:       Los Alamos Sea Ice Model, CICE_6.0.2
    comment:      This year has 365 days
    comment2:     File written on model date 19931231
    comment3:     seconds elapsed into model date:  54000
    conventions:  CF-1.0
    history:      This dataset was created on 2023-08-15 at 10:41:27.1
    io_flavor:    io_netcdf

xarray gives a nice view of the data, we can look at a single variable to see how the data are laid out

In [11]:
ds["hi"]

<xarray.DataArray 'hi' (time: 2, nj: 320, ni: 360)>
dask.array<concatenate, shape=(2, 320, 360), dtype=float32, chunksize=(1, 320, 360), chunktype=numpy.ndarray>
Coordinates: (12/26)
  * time     (time) datetime64[ns] 1993-12-31T15:00:00 1993-12-31T21:00:00
    TLON     (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    TLAT     (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ULON     (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ULAT     (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    NLON     (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ...       ...
    dxe      (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    dye      (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ANGLE    (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ANGLET   (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    cftime   (time) object 1993-12-31 15:00:00 1993-12-31 21:00:00
    ftime    (time) timedelta64[ns] -1 days +15:00:00 -1 days +15:00:00
Dimensions without coordinates: nj, ni
Attributes:
    units:          m
    long_name:      grid cell mean ice thickness
    cell_measures:  area: tarea
    comment:        ice volume per unit grid cell area
    time_rep:       instantaneous

## Regridding the dataset

We may want to regrid the dataset to a Gaussian grid utilized by FV3.

For this example, we will create a Gaussian grid for this purpose.

It is also possible to read the grid from an FV3 dataset using `RegridCICE6.read_grid()`

In [12]:
lats, lons = RegridCICE6.compute_gaussian_grid(180, 360)
lats

array([-89.23664167, -88.24777411, -87.25306675, -86.25703993,
       -85.26051261, -84.26374309, -83.26683823, -82.26985012,
       -81.2728072 , -80.27572627, -79.27861791, -78.28148911,
       -77.28434468, -76.28718801, -75.2900216 , -74.2928473 ,
       -73.29566652, -72.29848035, -71.30128967, -70.30409514,
       -69.30689734, -68.3096967 , -67.31249362, -66.31528838,
       -65.31808127, -64.32087249, -63.32366223, -62.32645066,
       -61.32923791, -60.3320241 , -59.33480934, -58.33759371,
       -57.3403773 , -56.34316017, -55.34594239, -54.34872402,
       -53.3515051 , -52.35428567, -51.35706578, -50.35984547,
       -49.36262476, -48.36540368, -47.36818227, -46.37096054,
       -45.37373851, -44.37651622, -43.37929366, -42.38207087,
       -41.38484786, -40.38762464, -39.39040122, -38.39317762,
       -37.39595384, -36.39872991, -35.40150582, -34.40428159,
       -33.40705723, -32.40983274, -31.41260814, -30.41538342,
       -29.41815859, -28.42093367, -27.42370865, -26.42

Create the Ocean dataset regridder object from the lats and lons

In [13]:
rg = RegridCICE6(lats, lons, ds, config_filename = "config-replay.yaml")

Regrid the whole dataset

In [14]:
ds = rg.regrid(ds)

In [15]:
ds

<xarray.Dataset>
Dimensions:         (time: 2, lon: 360, lat: 180)
Coordinates:
    cftime          (time) object 1993-12-31 15:00:00 1993-12-31 21:00:00
    ftime           (time) timedelta64[ns] -1 days +15:00:00 -1 days +15:00:00
  * lon             (lon) float64 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * lat             (lat) float64 -89.24 -88.25 -87.25 ... 87.25 88.25 89.24
  * time            (time) datetime64[ns] 1993-12-31T15:00:00 1993-12-31T21:0...
Data variables: (12/177)
    hi              (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    hs              (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    Tsfc            (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    aice            (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    uvel            (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    vvel            (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    ...              ...
    ipond_d         (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    apeff_d         (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    apond_h         (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    hpond_h         (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    ipond_h         (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    apeff_h         (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
Attributes:
    title:        sea ice model output for CICE
    contents:     Diagnostic and Prognostic Variables
    source:       Los Alamos Sea Ice Model, CICE_6.0.2
    comment:      This year has 365 days
    comment2:     File written on model date 19931231
    comment3:     seconds elapsed into model date:  54000
    conventions:  CF-1.0
    history:      This dataset was created on 2023-08-15 at 10:41:27.1
    io_flavor:    io_netcdf

## Storing the dataset

The dataset has more variables than we wanted, but these are trimmed when we call [.store_dataset()](generated/ufs2arco.CICE6Dataset.store_dataset.rst).

Any arguments beyond the dataset are passed to `xarray.to_zarr`.

By default, dask (which we're accessing through xarray under the hood) stores data to zarr using the chunksizes that are being used currently by dask (i.e., how the data are viewed in memory, currently).
However, we will change this layout based on the `chunks_out` option in our configuration yaml.

In [16]:
reader.chunks_out

{'time': 1, 'NCAT': 5, 'VGRDi': 7, 'VGRDa': 5, 'VGRDs': 5, 'nj': 30, 'ni': 30}

In [17]:
# overwrite chunks for regridded dataset
reader.chunks_out = {"time": 1, "NCAT": 5, "VGRDi": 7, "VGRDa": 5, "VGRDs":5, "lon": 30, "lat": 30}

<div class="alert alert-info">

Note

This CICE6 dataset has a number of static coordinates that are not dimensions of the data variables, but do not vary with time (e.g., `TLON`, `TLAT`, `NCAT`, etc). By default, [.store_dataset()](generated/ufs2arco.CICE6Dataset.store_dataset.rst) does not store these coordinates at all. However, we can pass `store_coords=True` in order to store the coordinates in a separate zarr store. This is done only once, and these coordinates are stored in a separate zarr store from the rest of the data variables.

We may change this, so that the coordinates are stored by default with the data variables.

</div>

In [18]:
reader.store_dataset(ds, store_coords=False, mode="w")

Stored dataset at example-replay-1deg/cice6.zarr


Now we can look at the coordinates

In [ ]:
cds = xr.open_zarr(reader.coords_path)

In [ ]:
cds

Which is separate from the data variables

In [ ]:
xds = xr.open_zarr(reader.data_path)

In [ ]:
xds

We can see that this only has the data variables we asked for in the config yaml file.

In [ ]:
xds.hi

And we can see this has a different chunking scheme than the original dataset, it has what we asked for in the config yaml.

In [ ]:
reader.chunks_out

<div class="alert alert-info">

Note:

This chunking scheme is not necessarily desirable, as the chunk sizes are quite small. This is simply used as an example to illustrate that we have control
over the chunk sizes.

</div>

## Cleanup

In [ ]:
from shutil import rmtree

In [ ]:
rmtree(reader.data_path)

In [ ]:
rmtree(reader.coords_path)